In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-24T14:43:06.281025+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')
WOLFRAM_ALPHA_APPID=config.get('General','WOLFRAM_ALPHA_APPID')

In [3]:
import langchain
import newspaper
import numpy
import openai
import sklearn
%watermark --iversions 

openai   : 1.3.2
langchain: 0.0.336
newspaper: 0.2.8
numpy    : 1.23.5
sklearn  : 1.3.2



## Recreating the Bing Chatbot

### Introduction

While the Large Language Models (LLMs) possess impressive capabilities, they have certain limitations that can present challenges when deploying them in a production environment. The hallucination problem makes them answer certain questions wrongly with high confidence. This issue can be attributed to various factors, one of which is that their training process has a cut-off date. So, these models do not have access to events preceding that date.

A workaround approach is to present the required information to the model and leverage its reasoning capability to find/extract the answer. Furthermore, it is possible to present the top-matched results a search engine returns as the context for a user’s query.

This lesson will explore the idea of finding the best articles from the Internet as the context for a chatbot to find the correct answer. We will use LangChain’s integration with Google Search API and the Newspaper library to extract the stories from search results. This is followed by choosing and using the most relevant options in the prompt.

<img src="https://images.spr.so/cdn-cgi/imagedelivery/j42No7y-dcokJuNgXeA0ig/43d1b24a-4834-4820-91f5-077c0b1e626e/Screenshot_2023-06-09_at_13.39.38/w=1920,quality=80"/>

In [4]:
# Get the token from Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['GOOGLE_CSE_ID'] = GOOGLE_CSE_ID

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
os.environ['ACTIVELOOP_TOKEN'] = ACTIVELOOP_TOKEN
my_activeloop_org_id = ACTIVELOOP_ORG_ID

The user query is used to extract relevant articles using a search engine (e.g. Bing or Google Search), which are then split into chunks. We then compute the embeddings of each chunk, rank them by cosine similarity with respect to the embedding of the query, and put the most relevant chunks into a prompt to generate the final answer, while also keeping track of the sources.

### Ask Trending Questions

Let’s start this lesson by seeing an example. The following piece must be familiar by now. It uses the OpenAI GPT-3.5-turbo model to create an assistant to answer questions. We will ask the model to name the latest Fast & Furious movie, released recently. Therefore, the model couldn’t have seen the answer during the training. 

In [5]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

template = """You are an assistant that answers the following question correctly and honestly: {question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)

question_chain = LLMChain(llm=llm, prompt=prompt_template)

question_chain.run("what is the latest fast and furious movie?")

'\nThe latest Fast and Furious movie is Fast & Furious 9, which was released in May 2021.'

The response shows that the model references the previous movie title as the answer. This is because the new movie (10th sequel) has yet to be released in its fictional universe! Let’s fix the problem.

### Get Search Results

This section uses LangChain’s GoogleSearchAPIWrapper class to receive search results. It works in combination with the Tool class that presents the utilities for agents to help them interact with the outside world. In this case, creating a tool out of any function, like top_n_results is possible. The API will return the page’s title, URL, and a short description.

In [6]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
TOP_N_RESULTS = 10

def top_n_results(query):
    return search.results(query, TOP_N_RESULTS)

tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=top_n_results
)

query = "What is the latest fast and furious movie?"

results = tool.run(query)

for result in results:
    print(result["title"])
    print(result["link"])
    print(result["snippet"])
    print("-"*50)

Fast & Furious movies in order | chronological and release order ...
https://www.radiotimes.com/movies/fast-and-furious-order/
Mar 22, 2023 ... Fast & Furious Presents: Hobbs & Shaw (2019); F9 (2021); Fast and Furious 10 (2023). Tokyo Drift also marks the first appearance of Han Lue, a ...
--------------------------------------------------
Fast & Furious - Wikipedia
https://en.wikipedia.org/wiki/Fast_%26_Furious
Fast X was designed to be the final movie of the franchise which later evolved in becoming a two part finale. Justin Lin was brought back to direct both movies ...
--------------------------------------------------
FAST X | Final Trailer - YouTube
https://www.youtube.com/watch?v=eoOaKN4qCKw
May 15, 2023 ... ... last 12 years masterminding a plan to make Dom pay the ultimate price ... Fast X, the tenth film in the Fast & Furious Saga, launches the ...
--------------------------------------------------
Here's How to Watch Every Fast and Furious Movie In Order ...
https://www.mens

Now, we use the results variable’s link key to download and parse the contents. The newspaper library takes care of everything. However, it might be unable to capture some contents in certain situations, like anti-bot mechanisms or having a file as a result.

In [7]:
import newspaper

pages_content = []

for result in results:
    try:
        article = newspaper.Article(result["link"])
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({ "url": result["link"], "text": article.text })
    except:
        continue

print(pages_content)

[{'url': 'https://www.radiotimes.com/movies/fast-and-furious-order/', 'text': 'With the two-part finale to the Fast saga wrapping up the story for Dominic Toretto (Vin Diesel) and his crew, the stakes are bound to be high.\n\nSo, ahead of Fast X, which will see the Toretto family tree branching out further, with Hollywood icon Rita Moreno joining the cast as Dom\'s grandmother, read on for everything you need to know about how watch the franchise in order.\n\nHow to watch the Fast and Furious movies: chronological timeline order\n\nAlthough most of the Fast films follow on from each other, there is one standout exception: you guessed it, The Fast and the Furious: Tokyo Drift.\n\nAlthough it was the third Fast film to be released in cinemas, Tokyo Drift is actually set between Fast & Furious 6 and Furious 7. Swerving away from the story of Brian and Dom, the film follows a completely new set of characters led by Sean Boswell (Lucas Black), a student forced to move to Japan to avoid jail

### Process the Search Results

We now have the top 10 results from the Google search. (Honestly, who looks at Google’s second page?) However, it is not efficient to pass all the contents to the model because of the following reasons:

    The model’s context length is limited.
    It will significantly increase the cost if we process all the search results.
    In almost all cases, they share similar pieces of information.

So, let’s find the most relevant results, 

Incorporating the LLMs embedding generation capability will enable us to find contextually similar content. It means converting the text to a high-dimensionality tensor that captures meaning. The cosine similarity function can find the closest article with respect to the user’s question.

It starts by splitting the texts using the RecursiveCharacterTextSplitter class to ensure the content lengths are inside the model’s input length. The Document class will create a data structure from each chunk that enables saving metadata like URL as the source. The model can later use this data to know the content’s location.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)

docs = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        new_doc = Document(page_content=chunk, metadata={ "source": d["url"] })
        docs.append(new_doc)
print(docs)

[Document(page_content="With the two-part finale to the Fast saga wrapping up the story for Dominic Toretto (Vin Diesel) and his crew, the stakes are bound to be high.\n\nSo, ahead of Fast X, which will see the Toretto family tree branching out further, with Hollywood icon Rita Moreno joining the cast as Dom's grandmother, read on for everything you need to know about how watch the franchise in order.\n\nHow to watch the Fast and Furious movies: chronological timeline order\n\nAlthough most of the Fast films follow on from each other, there is one standout exception: you guessed it, The Fast and the Furious: Tokyo Drift.\n\nAlthough it was the third Fast film to be released in cinemas, Tokyo Drift is actually set between Fast & Furious 6 and Furious 7. Swerving away from the story of Brian and Dom, the film follows a completely new set of characters led by Sean Boswell (Lucas Black), a student forced to move to Japan to avoid jail time.\n\nHere are the movies in chronological order:\n\

The subsequent step involves utilizing the OpenAI API's OpenAIEmbeddings class, specifically the .embed_documents() method for search results and the .embed_query() method for the user's question, to generate embeddings.

In [9]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

docs_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
query_embedding = embeddings.embed_query(query)

Lastly, the get_top_k_indices function accepts the content and query embedding vectors and returns the index of top K candidates with the highest cosine similarities to the user's request. Later, we use the indexes to retrieve the best-fit documents.

In [10]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
    # convert the lists of vectors to numpy arrays
    list_of_doc_vectors = np.array(list_of_doc_vectors)
    query_vector = np.array(query_vector)

    # compute cosine similarities
    similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

    # sort the vectors based on cosine similarity
    sorted_indices = np.argsort(similarities)[::-1]

    # retrieve the top K indices from the sorted list
    top_k_indices = sorted_indices[:top_k]

    return top_k_indices

top_k = 2
best_indexes = get_top_k_indices(docs_embeddings, query_embedding, top_k)
best_k_documents = [doc for i, doc in enumerate(docs) if i in best_indexes]
print(best_k_documents)

[Document(page_content='American media franchise\n\nFast & Furious (also known as The Fast and the Furious) is an American media franchise centered on a series of action films that are largely concerned with street racing, heists, spies, and family. The franchise also includes short films, a television series, toys, video games, live shows, and theme park attractions. The films are distributed by Universal Pictures.\n\nThe first film, based on the 1998 Vibe magazine article "Racer X" by Ken Li and written by Gary Scott Thompson, Erik Bergquist, and David Ayer, was released in 2001. It began the original tetralogy of films focused on illegal street racing, which culminated in the film Fast & Furious (2009). The series transitioned towards heists and spying with Fast Five (2011), which was followed by five sequels in that genre, the most recent of which, Fast X, was released on May 19, 2023.\n\nUniversal expanded the series to include the spin-off film Hobbs & Shaw (2019), while its subs

### Chain with Source

Finally, we used the selected articles in our prompt (using the stuff method) to assist the model in finding the correct answer. LangChain provides the load_qa_with_sources_chain() chain, which is designed to accept a list of input_documents as a source of information and a question argument which is the user’s question. The final part involves preprocessing the model’s response to extract its answer and the sources it utilized.

In [14]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

response = chain({"input_documents": best_k_documents, "question": query}, return_only_outputs=True)

response_text, response_sources = response["output_text"].split("SOURCES:")
response_text = response_text.strip()
response_sources = response_sources.strip()

In [16]:
print(f"Question: {query}")
print(f"Answer: {response_text}")
print(f"Sources: {response_sources}")

Question: What is the latest fast and furious movie?
Answer: Fast and Furious 10 (2023)
Sources: https://www.radiotimes.com/movies/fast-and-furious-order/
